In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
dir_path = "../../data/WHOI"
year = "2016"

In [7]:
def concatenate_WHOI_data(year, height_m):
    '''
    Concatenate the WHOI data from the given year at the given height measured
    param year : str : "2016", "2017", "2018", "2019"
    param height : int : 107, 187, etc
    '''
    dir_path = "../../data/WHOI"

    # iterate over data files that CSVs and concatenate
    files = os.listdir(os.path.join(dir_path, year))
    for i in range(len(files)):
        file = files[i]
        if file.endswith("csv"):
            if i == 0:
                data_file_df = pd.read_csv(os.path.join(dir_path, year, file)).dropna()[["Timestamp", "Position", "Temperature", f"{height_m}m Wind Direction (°)", f"{height_m}m Wind Speed (m/s)"]]
            else:
                try:
                    next_data_file_df = pd.read_csv(os.path.join(dir_path, year, file)).dropna()[["Timestamp", "Position", "Temperature", f"{height_m}m Wind Direction (°)", f"{height_m}m Wind Speed (m/s)"]]
                    # concatenate data dfs
                    data_file_df = pd.concat([data_file_df, next_data_file_df], axis=0)
                except:
                    print(f"Warning: skipping file #{i}, {file}")
                    continue
        # if i > 50:
        #     break
    
    # clean up some of the columns and add columns for t, wind direction, and height(°)
    data_file_df["Timestamp"] = pd.to_datetime(data_file_df.Timestamp, errors='coerce').dropna()
    data_file_df["t"] = data_file_df.Timestamp.values.astype(np.int64) // 10 ** 9
    # data_file_df["t"] = [timestamp.astype(np.int64)// 10 ** 9 if not pd.isnull(timestamp) else '' for timestamp in data_file_df['Timestamp']]

    data_file_df["wind_direction"] = data_file_df[f"{height_m}m Wind Direction (°)"]
    data_file_df["sine_wind_direction"] = np.sin(data_file_df["wind_direction"])
    data_file_df["z (m)"] = height_m

    # sort rows by date time
    data_file_df = data_file_df.sort_values(by="t").reset_index().drop(columns="index")
    min_t = data_file_df.iloc[0].Timestamp.month, data_file_df.iloc[0].Timestamp.day
    print(min_t)
    max_t = data_file_df.iloc[-1].Timestamp.month, data_file_df.iloc[-1].Timestamp.day
    
    # pickle final concatenated file
    final_concat_path = os.path.join(dir_path, f"{year}_z={height_m}m_{min_t[0],min_t[1]}-{max_t[0],max_t[1]}.pickle")
    
    with open(final_concat_path, "wb") as f:
        pickle.dump(data_file_df, f)
    
    return data_file_df

In [8]:
whoi_data_2016 = concatenate_WHOI_data(year="2018", height_m=107)

/var/folders/8s/cmks6vfx4vxb_x2clcgwn8080000gn/T/ipykernel_48823/2985724764.py:1: DtypeWarning: Columns (20,55,61,64,65,77,87) have mixed types.Specify dtype option on import or set low_memory=False.
  whoi_data_2016 = concatenate_WHOI_data(year="2018", height_m=107)


/var/folders/8s/cmks6vfx4vxb_x2clcgwn8080000gn/T/ipykernel_48823/2985724764.py:1: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  whoi_data_2016 = concatenate_WHOI_data(year="2018", height_m=107)
/var/folders/8s/cmks6vfx4vxb_x2clcgwn8080000gn/T/ipykernel_48823/2985724764.py:1: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  whoi_data_2016 = concatenate_WHOI_data(year="2018", height_m=107)
/var/folders/8s/cmks6vfx4vxb_x2clcgwn8080000gn/T/ipykernel_48823/2985724764.py:1: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  whoi_data_2016 = concatenate_WHOI_data(year="2018", height_m=107)
/var/folders/8s/cmks6vfx4vxb_x2clcgwn8080000gn/T/ipykernel_48823/2985724764.py:1: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  whoi_data_2016 = concatenate_WHOI_data(year="2018", height_m=107)


KeyboardInterrupt: 

In [35]:
whoi_data_2016

,Timestamp,Position,Temperature,107m Wind Direction (°),107m Wind Speed (m/s),t,wind_direction,sine_wind_direction,z (m)
0,2016-10-07 18:16:45.730,270,29.8,63.05,2.59,1475864205,63.05,0.216421,107
1,2016-10-07 18:16:47.450,0,29.8,63.54,2.59,1475864207,63.54,0.650427,107
2,2016-10-07 18:16:48.920,180,29.8,64.59,2.38,1475864208,64.59,0.982501,107
3,2016-10-07 18:16:48.180,90,29.8,63.38,2.38,1475864208,63.38,0.521107,107
4,2016-10-07 18:16:49.640,270,29.7,64.57,2.31,1475864209,64.57,0.986030,107
...,...,...,...,...,...,...,...,...,...
4531665,2016-12-30 23:59:55.430,90,11.0,225.66,13.73,1483142395,225.66,-0.509558,107
4531666,2016-12-30 23:59:56.160,180,11.0,226.12,13.69,1483142396,226.12,-0.074602,107
4531667,2016-12-30 23:59:56.890,270,11.0,227.54,14.74,1483142396,227.54,0.974690,107
4531668,2016-12-30 23:59:58.570,0,11.0,229.98,14.56,1483142398,229.98,-0.600188,107


## WHOI Data Analysis

In [ ]:
whoi_data_path = "../WHOI/2016_z=107m_(10, 7)-(12, 30).pickle"
with open(whoi_data_path, "rb") as f:
    whoi_data_df = pickle.load(whoi_data_path)
whoi_data_df

FileNotFoundError: [Errno 2] No such file or directory: 'data/WHOI/2016_z=107m_(10, 7)-(12, 30).pickle'